# Tarea. Notebook BoW
### Vladimir García Loginova

### Se importan paqueterías y se descargan elementos

In [1]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
nlp = spc.load("es_core_news_sm")


[nltk_data] Downloading package punkt to /Users/vcx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/vcx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Lectura de dataset

In [2]:
df=pd.read_csv("df_mini_HS.csv")

### Creación de listas con oraciones lematizadas y vocabulario de cada texto

In [3]:
all_voc=[]
ols=[]

for i in range(df.shape[0]):

    #linmpieza de datos
    oracion=df['text'].iloc[i]    
    oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
    oracion = re.sub("http[s]?\://\S+", "", oracion)  # Eliminar enlaces
    oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
    oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
    oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
    oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
    oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
    oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
    oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración 
    oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de exclamación
    oracion = re.sub(r"[-]", "", oracion)  # Eliminar guión
    oracion = re.sub(r"[:""´´''``]", "", oracion)  # Eliminar otros
    
    #tokenización
    tokens = word_tokenize(oracion)

    #stopwords
    spanish_stopwords = stopwords.words('spanish')
    palabras_filtradas = [palabra for palabra in tokens if palabra not in spanish_stopwords]

    #lematización
    lema = nlp(" ".join(palabras_filtradas))
    oracion_lematizada = " ".join([token.lemma_ for token in lema])
    vectorizador = CountVectorizer()
    vectores = vectorizador.fit_transform([oracion_lematizada])
    vocabulario = vectorizador.get_feature_names_out()

    #se guarda en listas las oraciones lematizadas y vocabulario de cada texto
    all_voc.append(vocabulario)
    ols.append(oracion_lematizada)

<>:9: SyntaxWarning: invalid escape sequence '\:'
<>:9: SyntaxWarning: invalid escape sequence '\:'
/var/folders/fw/lfwqgrwx72v8gqhx3vm2nv780000gn/T/ipykernel_852/4040966600.py:9: SyntaxWarning: invalid escape sequence '\:'
  oracion = re.sub("http[s]?\://\S+", "", oracion)  # Eliminar enlaces


### Vectores y vocabulario de todos los textos

In [4]:
# se crea un solo string de todas las oraciones lematizadas
oraciones_lem_pegadas=' '.join(ols)

#se crea el bag of words del vocabulario de todos los textos
vectorizador_all = CountVectorizer()
vectores_all = vectorizador_all.fit_transform([oraciones_lem_pegadas])
vocabulario_all = vectorizador_all.get_feature_names_out()
print(vocabulario_all)

['acoso' 'agar' 'agresión' 'al' 'amigo' 'amodio' 'analicemos' 'aprieto'
 'arab' 'arar' 'así' 'ayuda' 'bala' 'basta' 'bueno' 'bus' 'cabeza'
 'cadete' 'callate' 'calle' 'camello' 'cara' 'categoria' 'cerebro'
 'chavista' 'chiste' 'chorizo' 'ciento' 'ciudadanos' 'colaborador' 'colau'
 'conto' 'cortad' 'cosas' 'creer' 'cuatro' 'cuánto' 'cuántos' 'cállate'
 'de' 'decir' 'dejando' 'dejar' 'denunciaaaaaaar' 'des' 'detras' 'dia'
 'donbenitense' 'durar' 'eichmann' 'el' 'elite' 'emiratos' 'en'
 'encontrar' 'equipo' 'escolar' 'ese' 'españa' 'esperar' 'esperas'
 'estado' 'expulsarlos' 'falta' 'favor' 'femenino' 'festivit' 'frivolizar'
 'fuenlabrada' 'futbol' 'garcho' 'gente' 'guarra' 'hacer' 'hijo' 'hoy'
 'humillación' 'ilegal' 'inmigracion' 'inmigrante' 'inscrito'
 'josewifakers' 'juventud' 'kiev' 'ko' 'ladrón' 'llamado' 'luchar' 'lugar'
 'madre' 'malditar' 'mantener' 'mantero' 'marico' 'matar' 'medias' 'mejor'
 'metete' 'mientras' 'minato' 'morir' 'moro' 'mujer' 'mundo' 'nacional'
 'nazi' 'negrat

En el vocabulario global se observa que hay muchas palabras mal escritas o inexistentes en el español. En este caso, se dejan de esta forma con tal de no modificar el proceso. No obstante, es probable que se requiera un paso extra de limpieza de datos para tener un mejor BoW.

### Creación de Dataframe con bag of words de todos los textos
Las columnas 1 y 2 del Dataframe final corresponden al texto original y su oración lematizada, respectivamente. 
Las columnas 2 a 191 (longitud del vocabulario de todos los textos) corresponden al vector de cada texto con valores de 0 ó 1, si el texto contiene una palabra del vocabulario global el valor de la celda del vector es 1, caso contrario es 0.
La última columna corresponde al atributo de la clase (0 o 1) del texto original.

In [5]:
# Se crea matriz de ceros con dimensiones de número de textos x número de palabras del vocabulario global
final_vectors=np.zeros((df.shape[0],vocabulario_all.shape[0]),dtype=int)

# se sustituyen con 1 los indices donde se encuentra la palabra en el arreglo del vocabulario global
match_mask=[]
for ind,v in enumerate(all_voc):
    indexes=np.where(np.isin(vocabulario_all,v))[0]
    final_vectors[ind][indexes]=1

# se crea el dataframe final con los vectores por texto (cada fila corresponde a un texto)
df_bw = pd.DataFrame(final_vectors,columns = vocabulario_all)
#se inserta texto original
df_bw.insert(0,'texto',df['text'].values)
#se inserta oracion lematizada de cada texto
df_bw.insert(1,'oracion_lematizada', ols)
#se inserta atributo de cada clase en ultima columna
df_bw.insert(final_vectors.shape[1]+2,'label',df['label'].values)
#se imprime el dataframe del BoW final
print(df_bw)

                                                texto  \
0   ESAS COSAS Y OTRAS PUEDEN PASAR POR MANTENER A...   
1   28: te amodio, odio a la perra de tu amiga ☺️☺...   
2   @LaDivinaDiva Callate maldita perra. O seguro ...   
3   @MarysabelPuerto Mejor callate cara de puta o ...   
4   @xarita327 @TRIKYHUMOR @yonier2012 @casTa1326 ...   
5               @CocotePR @ashleyhonohan callate puta   
6   Y el inmigrante recibe ayuda del rico Estado l...   
7   De los moros no se puede esperar nada bueno, y...   
8   ¿Por que si a una mujer le pegan un tiro en la...   
9   Analicemos esto: ¿Si te pones unos shorts así,...   
10  "See... Tal vez les recordo como Peron protegi...   
11  PIETRAPIERCE STORY: PURS SANGS ARABES STARS DE...   
12  ¿Qué dice este de frivolizar el acoso escolar?...   
13  #Suiza 🇨🇭 retira el permiso a 189 refugiados q...   
14  Hoy quiero denunciaaaaaaar A LA GENTE PUTO GUA...   
15  Redomicilie su sociedad offshore en Emiratos Á...   
16  @hermanntertsch Basta! Pone